In [1]:
import torch
import functions
import modules
import math
from optimizer import SGD

from torch import optim
from torch import Tensor
from torch import nn

# torch.set_grad_enabled(False)

- Generates a training and a test set of 1,000 points sampled uniformly in [0, 1]2, each with a
label 0 if outside the disk of radius 1/√2π and 1 inside,
- two input units, two output units, three hidden layers of 25 units,

## Test.py

## Generate data

In [2]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    target = input.pow(2).sum(1).sub(1 / (2 * math.pi)).sign().add(1).div(2).long()
    return input, target

In [3]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [4]:
train_input, train_target = generate_disc_set(3)

In [5]:
print(train_input)
print(train_target)

tensor([[0.9600, 0.9407],
        [0.0746, 0.6593],
        [0.6379, 0.0767]])
tensor([1, 1, 1])


In [6]:
train_target = convert_to_one_hot_labels(train_input, train_target)

In [7]:
print(train_input)
print(train_target)

tensor([[0.9600, 0.9407],
        [0.0746, 0.6593],
        [0.6379, 0.0767]])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.]])


In [8]:
train_input, train_target = generate_disc_set(1000)
train_target = convert_to_one_hot_labels(train_input, train_target)

In [9]:
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)

tensor([[-1.6074, -0.9145],
        [-0.9288,  0.8357],
        [ 0.5624,  1.2244],
        ...,
        [ 1.2717, -1.6148],
        [ 0.7376, -0.5644],
        [-0.8719, -0.5701]])

In [10]:
train_input = 0.9 * train_input

In [11]:
print(train_input.shape)
print(train_target.shape)

torch.Size([1000, 2])
torch.Size([1000, 2])


In [18]:
arch1 = modules.Sequential(modules.Linear(2, 25), modules.Linear(25, 2), modules.MSELoss(train_target))

In [19]:
for name, m in arch1.__dict__.items():
    print(name)
    print(m)

Linear0
Linear1
MSELoss2


In [20]:
nb_epochs = 2
lr = 1e-3
optimizer = SGD(arch1.param(), lr=lr)

for e in range(nb_epochs):
    output, loss = arch1.forward(train_input)
    grad = arch1.backward()
    for (p, grad2) in arch1.param():
        p -= lr * grad2
        grad2 = torch.zeros(grad2.shape)

tensor(1957.7909)
tensor(31735.6914)


In [ ]:
model = nn.Sequential(
            nn.Linear(2, 2),
#             nn.ReLU(),
#             nn.Linear(128, 2)
        )
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
nb_epochs = 10

for e in range(nb_epochs):
    output = model(train_input)
    print(output)
    loss = criterion(output, train_target.float())
#     print(loss)
    model.zero_grad()
    loss.backward()
    optimizer.step()

## Mini-Project Architecture

In [ ]:
seq = modules.Sequential(modules.Linear(2, 25),
                         modules.TanH(),
                         modules.Linear(25, 25),
                         modules.TanH(),
                         modules.Linear(25, 25),
                         modules.TanH(),
                         modules.Linear(25, 25),
                         modules.TanH(),
                         modules.Linear(25, 2)
                        )